In [38]:
import os
from dotenv import load_dotenv
from src.data.rag import retrieve_relevant_context
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.tools import tool


llm = ChatOllama(
    model="gemma3",
)

In [39]:
@tool
def submit_ticket(name: str, email: str) -> None:
    """Submits a Jira Ticket once all required information is passed"""
    print("Hooray, a ticket has been created for {name} with email {email}")

llm.bind_tools([submit_ticket])

RunnableBinding(bound=ChatOllama(model='gemma3'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'submit_ticket', 'description': 'Submits a Jira Ticket once all required information is passed', 'parameters': {'properties': {'name': {'type': 'string'}, 'email': {'type': 'string'}}, 'required': ['name', 'email'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [40]:
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods.
Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email. Once you have their name and email, you can create Jira Ticket using functions

You have access to functions. If you decide to invoke any of the function(s), you MUST put it in the format of
[func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]

You SHOULD NOT include any other text in the response if you call a function
[
  {
    "name": "submit_ticket",
    "description": "Submits a Jira Ticket once all required information is passed",
    "parameters": {
      "type": "object",
      "properties": {
        "name": {
          "type": "string"
        },
        "email": {
          "type": "string"
        }
      },
      "required": [
        "name",
        "email"
      ]
    }
  }
]


DO NOT MAKE UP OR SIMULATE INFORMATION.
""".replace("{", "{{").replace("}", "}}")

question = """
Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [42]:
output = chain.invoke({"question": question,})
output

AIMessage(content='[submit_ticket(name="Harsh", email="patil@gmail.com")]', additional_kwargs={}, response_metadata={'model': 'gemma3', 'created_at': '2025-04-15T04:27:24.588369491Z', 'done': True, 'done_reason': 'stop', 'total_duration': 762345564, 'load_duration': 56928887, 'prompt_eval_count': 325, 'prompt_eval_duration': 38407237, 'eval_count': 20, 'eval_duration': 665215670, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-023e0d84-043d-40cb-985b-8d5ab3ed866c-0', usage_metadata={'input_tokens': 325, 'output_tokens': 20, 'total_tokens': 345})

In [44]:
import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

# 1. Define the tool
@tool
def submit_ticket(name: str, email: str) -> str:
    """Submits a Jira Ticket once all required information is passed"""
    print(f"Hooray, a ticket has been created for {name} with email {email}")
    return f"Ticket created for {name} with {email}"

# 2. Use ChatOllama directly (no .with_tools)
llm = ChatOllama(model="gemma3")

# 3. Prompt construction
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods.
Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email. Once you have their name and email, you can create Jira Ticket using functions.

You have access to functions. If you decide to invoke any of the function(s), you MUST put it in the format of
[submit_ticket(name="...", email="...")]
You SHOULD NOT include any other text in the response if you call a function.
""".replace("{", "{{").replace("}", "}}")

question = "Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])

# 4. Run the LLM
chain = prompt | llm
response = chain.invoke({"question": question})

# 5. Manually parse for function call
match = re.search(r'submit_ticket\(name="(.+?)", email="(.+?)"\)', response.content)
if match:
    name = match.group(1)
    email = match.group(2)
    result = submit_ticket.invoke({"name": name, "email": email})
    print("Tool Output:", result)
else:
    print("No tool call detected.")


Hooray, a ticket has been created for Harsh with email patil@gmail.com
Tool Output: Ticket created for Harsh with patil@gmail.com


In [46]:
import os
from dotenv import load_dotenv
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

# Set up the LLM
llm = ChatOllama(
    model="mistral-small",
)

@tool
def submit_ticket(name: str, email: str) -> str:
    """Submits a Jira Ticket once all required information is passed"""
    print(f"Hooray, a ticket has been created for {name} with email {email}")
    return f"Successfully created a ticket for {name} with email {email}"

# Bind tools to the LLM
llm_with_tools = llm.bind_tools([submit_ticket])

# Modified system prompt to use standard tool calling format
system_prompt = """
You are a frontdesk assistant for Capital Area Food Bank (CAFB).

A partner is creating a new issue of type: Delivery Issues/Damaged Goods. Help them by gathering more information about their issue so that a Jira Ticket can be created. You need to get their name and email.

Once you have their name and email, you MUST use the submit_ticket tool to create a Jira Ticket.
"""

question = """
Hello, I need help with my damaged goods. My name is Harsh and my email is patil@gmail.com. Please help me
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}"),
])

# Create the chain with the LLM that has tools bound to it
chain = prompt | llm_with_tools

# Run the chain
output = chain.invoke({"question": question})
print(output)

content='' additional_kwargs={} response_metadata={'model': 'mistral-small', 'created_at': '2025-04-15T04:31:15.298583717Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22079905837, 'load_duration': 10600946065, 'prompt_eval_count': 188, 'prompt_eval_duration': 1528294533, 'eval_count': 28, 'eval_duration': 9945355378, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-e8d723e7-958c-489a-b163-269709e0f65b-0' tool_calls=[{'name': 'submit_ticket', 'args': {'email': 'patil@gmail.com', 'name': 'Harsh'}, 'id': '623098d6-d5ba-4334-94b9-47287d7ccd7e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 188, 'output_tokens': 28, 'total_tokens': 216}


In [48]:
output.tool_calls

[{'name': 'submit_ticket',
  'args': {'email': 'patil@gmail.com', 'name': 'Harsh'},
  'id': '623098d6-d5ba-4334-94b9-47287d7ccd7e',
  'type': 'tool_call'}]

In [9]:
from openai import OpenAI
import os
client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY')
)
models = client.models.list()
for model in models:
    print(model.id)

gpt-4o-audio-preview-2024-12-17
dall-e-3
text-embedding-3-large
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4.1-mini
gpt-4.1-mini-2025-04-14
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview
babbage-002
tts-1-hd-1106
gpt-4
text-embedding-ada-002
tts-1-hd
gpt-4o-mini-audio-preview
gpt-4o-audio-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4-0125-preview
gpt-4o-mini-search-preview
gpt-4-turbo-preview
tts-1-1106
chatgpt-4o-latest
davinci-002
gpt-3.5-turbo-1106
gpt-4-turbo
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-2024-11-20
whisper-1
gpt-4.1
gpt-4.1-2025-04-14
gpt-4o-2024-05-13
gpt-3.5-turbo-16k
gpt-4-turbo-2024-04-09
gpt-4-1106-preview
o1-preview
gpt-4-0613
gpt-4o-search-preview
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
tts-1
omni-moderation-2024-09-2

In [ ]:
from langchain_openai import ChatOpenAI
from src.data.jira import get_issue_context, get_issue_kb
from langchain_core.runnables.history import RunnableWithMessageHistory
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


llm = ChatOpenAI(model='gpt-3.5-turbo-instruct')

issue_context = get_issue_context('CAFBSS-456').replace("{", "{{").replace("}", "}}")
issue_kb = get_issue_kb('CAFBSS-456').replace("{", "{{").replace("}", "}}")
system_prompt = f"""
You are a frontdesk assistant for Capital Area Food Bank (CAFB). 
A partner is inquiring about an existing ticket with ID: CAFBSS-456.

Try to provide helpful context and solutions related to this ticket.
Use the Ticket Context and Knowledge Base to answer questions
If the answer cannot be found in the context or if no context is given, say so clearly and suggest how the user might refine their question. 
DO NOT MAKE UP OR SIMULATE INFORMATION.

Ticket Context: {issue_context}
    
##############
Knowledge Base: {issue_kb}
"""

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: msgs,
    input_messages_key="question",
    history_messages_key="history",
)

In [3]:
from src.data.db import connect_to_db
from src.data.jira import connect_to_jira

def close_ticket(issue_key: str, resolution_reason: str, summary: str = None):
    """
    Use this when the partner's issue has been resolved and they confirm the ticket can be closed. Input should be the resolution reason.
    """
    jira = connect_to_jira()
    
    try:
        issue = jira.issue(issue_key)
        
        # Add summary comment if provided
        if summary:
            jira.add_comment(issue, f"Final Summary:\n\n{summary}")
        
        # Add resolution comment
        jira.add_comment(issue, f"Resolution: {resolution_reason}")
        
        # Close the ticket - transition the issue to the "Done" status
        # Note: The exact transition ID/name may vary based on your JIRA workflow
        transitions = jira.transitions(issue)
        close_transition = None
        
        # Find the right transition ID for closing the issue
        for t in transitions:
            if t['name'].lower() in ['close', 'done', 'resolve', 'closed', 'complete']:
                close_transition = t['id']
                break
        
        if close_transition:
            jira.transition_issue(
                issue,
                close_transition
            )
            
            # Update the database record
            conn = connect_to_db()
            cursor = conn.cursor()
            cursor.execute(
                """
                UPDATE jira_issues
                SET status = 'Closed', resolved_at = NOW(), updated_at = NOW()
                WHERE issue_key = %s
                """,
                (issue_key,)
            )
            conn.commit()
            conn.close()
            
            return {
                "status": 200,
                "display": f"✓ Ticket {issue_key} has been closed successfully"
            }
        else:
            return {
                "status": 400,
                "display": f"❌ Could not find a transition to close the ticket"
            }
        
    except Exception as e:
        return {
            "status": 500,
            "display": f"❌ Failed to close ticket: {str(e)}"
        }
    

2025-04-17 10:09:17.726 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/harshavardhan-patil/Work/Projects/cafb-ai


In [5]:
close_ticket('CAFBSS-1027', "All good", "The Partner has indicated that the issue is resolved")

{'status': 200, 'display': '✓ Ticket CAFBSS-1027 has been closed successfully'}

In [6]:
def update_ticket_priority(issue_key: str, new_priority: str, reason: str):
    """
    Use this to update the priority of a JIRA ticket. For escalations use priority High
    """
    jira = connect_to_jira()
    
    try:
        issue = jira.issue(issue_key)
        
        # Directly update the priority field
        issue.update(fields={'priority': {'name': new_priority}})
        
        # Add comment explaining the escalation
        comment_body = f"Priority escalated to {new_priority}.\nReason: {reason}"
        jira.add_comment(issue, comment_body)
        
        # Update the database record
        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(
            """
            UPDATE jira_issues
            SET priority = %s, updated_at = NOW()
            WHERE issue_key = %s
            """,
            (new_priority, issue_key)
        )
        conn.commit()
        conn.close()
        
        return f"✓ Ticket {issue_key} priority updated to {new_priority}"
        
    except Exception as e:
        return {
            "status": 500,
            "display": f"❌ Failed to update ticket priority: {str(e)}"
        }


In [7]:
update_ticket_priority("CAFBSS-1029", "High", "Cuz work fast pliss")

'✓ Ticket CAFBSS-1029 priority updated to High'